# Midterm 2021



In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from arch import arch_model
from arch.univariate import GARCH, EWMAVariance 
from sklearn import linear_model
import scipy.stats as stats
from statsmodels.regression.rolling import RollingOLS
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.precision", 4)
sns.set(rc={'figure.figsize':(15, 10)})

# 1

False, if the factors work perfectly, then together they span the tangency portfolio. It Just means a portfolio of the three factors has the maximum sharpe ratio because alpha is 0, but does not say anything about their individual Sharpe Ratios. 

# 2

False a stock with higher investment beta holding other factors fixed actually hurts returns. 

# 3

If stock was correlated to each Famma-French factor then the model would predict nothing about the mean return of this stock.

# 4

To ensure the momentum factor does not have too much idiosyncratic risk it needs to be both long-short to make net-investment 0 and to limit turnover one must increase the time frame of 'winners' and 'losers'. 

# 5

If the CAPM were true, 

The time-series R-squareds provide no useful interpretation because it explains how well the factors explain portfolio returns **not how well the factor betas explain portfolio returns.**

The information ratios would be zero because the CAPM model assume no-arbitrage of pricing. 

# 8

- Yes, in the long-run the market return will almost surely outperform the risk-free rate. 
- Due to the law of large numbers, this is obvious. Volatility increases at the square root of time, however, expected returns increase by a multiple of time. 

- This is obviously false because the volatility of 100 year cumulative return grows with the horizon. 

# Part 2

Use the factors in the data to test a two-factor pricing model. The factors are the market and crude oil factor. 

- Calculate the time-series regression of the commodity assets on the factors. 

- For the asset NG1, report the alpha and betas of the time-series regression. 



In [73]:
# Commodities 
file_path = 'C:/Users/dcste/OneDrive/Portfolio_Theory/Homework_Jupyter/portfolio_theory/midterm_2_data_pricing.xlsx'
commodities = pd.read_excel(file_path, sheet_name= 'assets (excess returns)').set_index("Date")
factors = pd.read_excel(file_path, sheet_name = 'factors (excess returns)').set_index("Date")
factors.head()

,MKT,CL1
Date,,
2000-01-31,-0.0474,0.0756
2000-02-29,0.0245,0.0966
2000-03-31,0.0520,-0.1207
2000-04-30,-0.0640,-0.0477
2000-05-31,-0.0442,0.1220


In [74]:
# Calculating time_series regressions

ts_regression_df = pd.DataFrame(data = None, index = commodities.columns, columns = ['Alpha', 'MKT','CL1'])

for asset in commodities.columns:
    X = sm.add_constant(factors)
    y = commodities[asset]
    regress = sm.OLS(y,X).fit().params
    ts_regression_df.loc[asset]  = [regress[0]*12, regress[1], regress[2]]
ts_regression_df    
    

,Alpha,MKT,CL1
NG1,0.1195,-0.0377,0.2502
KC1,0.0203,0.2992,0.0321
CC1,0.0632,0.1139,0.1243
LB1,0.0555,0.7791,0.1874
CT1,0.013,0.5291,0.0629
SB1,0.0696,0.0579,0.1628
LC1,0.0163,0.1068,0.0529
W1,0.0558,0.2912,-0.0026
S1,0.0421,0.3533,0.0386
C1,0.0609,0.2551,0.0652


# 2.1 

For asset NG1, report the alpha and betas of the regression

In [75]:
ts_regression_df.loc['NG1'].to_frame("NG1 Premia")

,NG1 Premia
Alpha,0.1195
MKT,-0.0377
CL1,0.2502


# 2.2 

Report the two-factor premia implied by the time-series test. Annualize them. 


In [76]:
(factors.mean()*12).to_frame('Factor Premia')

,Factor Premia
MKT,0.0707
CL1,0.1087


# 2.3

- Report he MAE of the model and annualize it

In [77]:
print("The MAE of the model is ", round(ts_regression_df['Alpha'].abs().mean(),4))

The MAE of the model is  0.0549


# 2.4

- Report the largest predicted premium from the model and note wich asset it is

In [80]:
(factors.mean()*12 *ts_regression_df[['MKT','CL1']]).sum(axis = 1).to_frame('Predicted Premia').nlargest(3, "Predicted Premia")

,Predicted Premia
LB1,0.0754
HG1,0.0608
PA1,0.0476


LB1 has the largest predicted premia of 7.5% according to the the time series regression

# 3 Pricing Model 

- For the cross-sectional regression report the r-squared and alpha (annualize alpha)
  

In [86]:
X_cs = sm.add_constant(ts_regression_df[['MKT','CL1']]).astype('float')
y = commodities.mean()
cs_reg = sm.OLS(y,X_cs).fit()

In [87]:
cs_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.631
Model:                            OLS   Adj. R-squared:                  0.564
Method:                 Least Squares   F-statistic:                     9.417
Date:                Mon, 07 Nov 2022   Prob (F-statistic):            0.00414
Time:                        16:53:38   Log-Likelihood:                 70.426
No. Observations:                  14   AIC:                            -134.9
Df Residuals:                      11   BIC:                            -132.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0038      0.001      3.632      0.004       0.001       0.006
MKT            0.0015      0.002      0.748      0.470      -0.003       0.006
CL1            0.0277      0.006      4.264      0.001       0.013       0.042
==============================================================================
Omnibus:                        0.783   Durbin-Watson:                   2.090
Prob(Omnibus):                  0.676   Jarque-Bera (JB):                0.636
Skew:                          -0.068   Prob(JB):                        0.728
Kurtosis:                       1.965   Cond. No.                         14.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [92]:
cs_regression_df = pd.DataFrame(data = None, index = ['Alpha', "MKT Beta",'Oil Beta','Rsquared','Tracking Error'], columns = ['Cross Sectional Test'])

cs_regression_df.loc['Alpha'] = cs_reg.params[0]*12
cs_regression_df.loc['MKT Beta'] = cs_reg.params[1]*12
cs_regression_df.loc['Oil Beta'] = cs_reg.params[2]*12
cs_regression_df.loc['Rsquared'] = cs_reg.rsquared
cs_regression_df.loc['Tracking Error'] = cs_reg.resid.abs().mean()*12

cs_regression_df

,Cross Sectional Test
Alpha,0.0456
MKT Beta,0.0186
Oil Beta,0.3319
Rsquared,0.6313
Tracking Error,0.0169


Yes, if the two-factor model accurately priced the commodity assets, alpha would be zero and the r-squared would be equal to 1.

# 3.4 

Report the largest predicted premium of the model, and note which asset it is.
- This is finished


In [99]:
predicted = cs_reg.params[0] + (ts_regression_df[['MKT','CL1']]*cs_reg.params[1:]).sum(axis = 1)
(predicted.sort_values(ascending = False)*12).to_frame('Largest Predicted Premia')

,Largest Predicted Premia
NG1,0.1279
LB1,0.1223
SI1,0.1207
HG1,0.1108
SB1,0.1007
PA1,0.0979
CC1,0.0890
CT1,0.0763
C1,0.0720
GC1,0.0668


# 4.1

The r-squared is more useful from the cross sectional regression because it explains the percent variation in the returns of the portfolios are explained by the betas. The time series r-squared does not provide a meaningful interpretation with respect to the predictive power of the beta. 

# 4.2

- If we include an intercept in the cross-sectional estimation then the MAE is always smaller than the time-series MAE. 

# 4.3

- If we add another factor this does not mean the time-series MAE will decrease because this factor could do a terrible job at explaining the commodity prices. However, the cross-sectional MAE will certainly decrease because you are adding another factor beta to the model, which objectively always increases the r-squared. 

In [100]:
def tangency_portfolio_rfr(asset_return,cov_matrix):
    """ 
        Returns the tangency portfolio weights in a (1 x n) vector when a riskless assset is available
        Inputs: 
            asset_return - Excess return over the risk free rate for each asset (n x 1) Vector
            cov_matrix = nxn covariance matrix for the assets
    """
    asset_cov = np.array(cov_matrix)
    inverted_cov= np.linalg.inv(asset_cov)
    one_vector = np.ones(len(cov_matrix.index))
    
    den = (one_vector @ inverted_cov) @ (asset_return)
    num =  inverted_cov @ asset_return
    return (1/den) * num

In [105]:
tang_factors = tangency_portfolio_rfr(factors.mean(),factors.cov())
tang_factors = pd.DataFrame(tang_factors, index = ['MKT','CL1'], columns = ['Tangency Weights'])

In [106]:
tang_factors

,Tangency Weights
MKT,0.8811
CL1,0.1189


CL1 does not receive that much weight relative to the MKT portfolio. Conceptually, this means it is a much less useful pricing factor than the market. We expect the weight to have a larger weight because investors would move in to these two diversifiable assets to extract the premium. 